In [21]:
import torch 
import numpy as np
import ipytest
ipytest.autoconfig()

In [22]:
def Conv3D(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'):
  def convolution(matrix):

    #groups exeption
    if (in_channels%groups != 0) or (out_channels%groups!=0):
      raise Exception('in_channels and out_channels must be divisible by groups')

    # Генерация смещения (bias)
    if bias:
      bias_values = torch.rand(out_channels)
    else:
      bias_values = torch.zeros(out_channels)

    # Режим заполнения (padding_mode)
    if padding_mode == 'zeros':
      pad = torch.nn.ZeroPad2d(padding)
      matrix = pad(matrix)
    elif padding_mode == 'reflect':
      pad = torch.nn.ReflectionPad2d(padding)
      matrix = pad(matrix)
    elif padding_mode == 'replicate':
      pad = torch.nn.ReplicationPad2d(2)
      matrix = pad(matrix)
    elif padding_mode == 'circular':
      pad = torch.nn.CircularPad2d(padding)
      matrix = pad(matrix)

    #генерация ядра
    if type(kernel_size) == tuple:
      filter = torch.rand(out_channels, in_channels//groups, kernel_size[0], kernel_size[1], kernel_size[2])
    if type(kernel_size) == int:
      filter = torch.rand(out_channels, in_channels//groups, kernel_size, kernel_size, kernel_size)


    convolution = []
    for l in range(out_channels):

      feature_map = np.array([])  # Инициализация пустой карты признаков

      # Цикл по ширине входной карты признаков с учетом размера фильтра, дилатации и шага
      # Определяет начальную позицию фильтра вдоль ширины входной карты признаков
      # Диапазон гарантирует, что фильтр остается в пределах размеров входной карты признаков
      for k in range (0, matrix.shape[1]-((filter.shape[2]-1)*dilation+1)+1, stride):
      
        # Цикл по высоте входной карты признаков с учетом размера фильтра, дилатации и шага
        # Определяет начальную позицию фильтра вдоль высоты входной карты признаков
        # Диапазон гарантирует, что фильтр остается в пределах размеров входной карты признаков
        for i in range (0, matrix.shape[2]-((filter.shape[3]-1)*dilation+1)+1, stride):

          # Цикл по глубине входной карты признаков с учетом размера фильтра, дилатации и шага
          # Определяет начальную позицию фильтра вдоль глубины входной карты признаков
          # Диапазон гарантирует, что фильтр остается в пределах размеров входной карты признаков
          for j in range (0, matrix.shape[3]-((filter.shape[4]-1)*dilation+1)+1, stride):
            
            # Инициализация переменной для хранения суммы сверток для текущей позиции
            total = 0
            for c in range (in_channels//groups): 
              # Выбор соответствующего среза входной матрицы для операции свертки
              if groups>1:
                val = matrix[l*(in_channels//groups)+c][k:k+(filter.shape[2]-1)*dilation+1:dilation, i:i+(filter.shape[3]-1)*dilation+1:dilation, j:j+(filter.shape[4]-1)*dilation+1:dilation]
              else:
                # Извлекаем подматрицу из входной матрицы matrix для одного канала (c)
                # с использованием трехмерного среза по ширине, высоте и глубине
                # Срез происходит вдоль каждого измерения с шагом dilation
                # Начальные индексы для каждого измерения определяются переменными k, i, j
                # и учитывают размеры фильтра и дилатацию
                val = matrix[c][k:k+(filter.shape[2]-1)*dilation+1:dilation, i:i+(filter.shape[3]-1)*dilation+1:dilation, j:j+(filter.shape[4]-1)*dilation+1:dilation]
              
              # Подсчет суммы элементов после умножения на веса фильтра
              local_sum = (val*filter[l][c]).sum()
              total = total + local_sum

            feature_map = np.append(feature_map, float(total + bias_values[l])) 

      # Добавляем результат свертки для каждого канала и позиции фильтра в массив convolution
      convolution.append(feature_map.reshape(
        (matrix.shape[1]-((filter.shape[2]-1)*dilation+1))//stride+1,   #по ширине
        (matrix.shape[2]-((filter.shape[3]-1)*dilation+1))//stride+1,   #по высоте
        (matrix.shape[3]-((filter.shape[4]-1)*dilation+1))//stride+1))  #по глубине

    return np.array(convolution), torch.tensor(np.array(filter)), torch.tensor(np.array(bias_values))

  return convolution

In [23]:
def create_random_tensor(shape):
    return torch.rand(shape)

def compare_convolutions(custom_conv, torch_conv, input_tensor):
    result, kernel, bias_val = custom_conv(input_tensor)
    torch_conv.weight.data = torch.tensor(kernel)
    torch_conv.bias.data = torch.tensor(bias_val)

    custom_result = str(np.round(result, 2))
    torch_result = str(np.round(np.array(torch_conv(input_tensor).data), 2))

    assert torch_result == custom_result

In [24]:
def test_1():
    tensor = create_random_tensor((10, 28, 28, 28))
    conv = Conv3D(               in_channels=10, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
    torch_conv = torch.nn.Conv3d(in_channels=10, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
    compare_convolutions(conv, torch_conv, tensor)

def test_2():
    tensor = create_random_tensor((3, 12, 12, 10))
    conv = Conv3D(               in_channels=3, out_channels=1, kernel_size=4, stride=2, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
    torch_conv = torch.nn.Conv3d(in_channels=3, out_channels=1, kernel_size=4, stride=2, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
    compare_convolutions(conv, torch_conv, tensor)

def test_3():
    tensor = create_random_tensor((4, 4, 4, 4))
    conv = Conv3D(               in_channels=4, out_channels=4, kernel_size=1, stride=1, padding=0, dilation=1, groups=4, bias=True, padding_mode='zeros')
    torch_conv = torch.nn.Conv3d(in_channels=4, out_channels=4, kernel_size=1, stride=1, padding=0, dilation=1, groups=4, bias=True, padding_mode='zeros')
    compare_convolutions(conv, torch_conv, tensor)
    
def test_4():
    tensor = create_random_tensor((3, 32, 32, 32))
    conv = Conv3D(               in_channels=3, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
    torch_conv = torch.nn.Conv3d(in_channels=3, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
    compare_convolutions(conv, torch_conv, tensor)

In [25]:
ipytest.run()

....                                                                                         [100%]
======================================== warnings summary =========================================
t_b2fd461f424a460ea3d4d79997e1b9f4.py::test_1
t_b2fd461f424a460ea3d4d79997e1b9f4.py::test_2
t_b2fd461f424a460ea3d4d79997e1b9f4.py::test_3
t_b2fd461f424a460ea3d4d79997e1b9f4.py::test_4
  C:\Users\Никола\AppData\Local\Temp\ipykernel_29120\69101569.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
    torch_conv.weight.data = torch.tensor(kernel)

t_b2fd461f424a460ea3d4d79997e1b9f4.py::test_1
t_b2fd461f424a460ea3d4d79997e1b9f4.py::test_2
t_b2fd461f424a460ea3d4d79997e1b9f4.py::test_3
t_b2fd461f424a460ea3d4d79997e1b9f4.py::test_4
  C:\Users\Никола\AppData\Local\Temp\ipykernel_29120\69101569.py:7: UserWarning: To copy construct from a tensor, it 

<ExitCode.OK: 0>